In [1]:
import numpy as np 
import pandas as pd
import time
start_time = time.time()

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import pipeline, grid_search
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
from nltk.stem.porter import *
stemmer = PorterStemmer()
import re
import random
from collections import Counter

C:\Users\Karthik\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Karthik\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
from sklearn.metrics import mean_squared_error, make_scorer

In [4]:

random.seed(2016)
df_train = pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/test.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/attributes.csv')
df_pro_desc = pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/product_descriptions.csv')

In [8]:
df_train.shape

(74067, 5)

In [7]:
df_test.shape

(166693, 4)

In [6]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})

In [7]:
num_train = df_train.shape[0]
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description,brand
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...,Delta
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...,Delta


In [8]:
num_train = df_train.shape[0]

In [9]:
def str_stem(s): 
    if isinstance(s, str):
        s = s.lower()
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) ##'desgruda' palavras que estão juntas

        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
    
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
    
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
    
        s = s.replace(" x "," xby ")
        s = s.replace("*"," xby ")
        s = s.replace(" by "," xby")
        s = s.replace("x0"," xby 0")
        s = s.replace("x1"," xby 1")
        s = s.replace("x2"," xby 2")
        s = s.replace("x3"," xby 3")
        s = s.replace("x4"," xby 4")
        s = s.replace("x5"," xby 5")
        s = s.replace("x6"," xby 6")
        s = s.replace("x7"," xby 7")
        s = s.replace("x8"," xby 8")
        s = s.replace("x9"," xby 9")
        s = s.replace("0x","0 xby ")
        s = s.replace("1x","1 xby ")
        s = s.replace("2x","2 xby ")
        s = s.replace("3x","3 xby ")
        s = s.replace("4x","4 xby ")
        s = s.replace("5x","5 xby ")
        s = s.replace("6x","6 xby ")
        s = s.replace("7x","7 xby ")
        s = s.replace("8x","8 xby ")
        s = s.replace("9x","9 xby ")
        
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
    
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
    
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
    
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
    
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
    
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        
        
        
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool")
        s = s.replace("whirlpoolstainless","whirlpool stainless")

        s = s.replace("  "," ")
        #s = (" ").join([stemmer.stem(z) for z in s.lower().split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        return s.lower()
    else:
        return "null"

In [10]:
def stopper(s):
    return " ".join([w for w in s.split() if w not in stop])

In [11]:

def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [ ]:

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
df_all['search_term'] = df_all['search_term'].map(lambda x: correction(x))
df_all['product_title'] = df_all['product_title'].map(lambda x: correction(x))
df_all['product_description'] = df_all['product_description'].map(lambda x: correction(x))


In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df_all['search_term'] = df_all['search_term'].map(lambda x:stopper(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:stopper(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:stopper(x))
df_all['brand'] = df_all['brand'].astype(str)
df_all['brand'] = df_all['brand'].map(lambda x:stopper(x))
#type(df_all['brand'])

In [15]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))

In [16]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)

In [17]:
df_brand = pd.unique(df_all.brand.ravel())

In [18]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']
#df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))

#df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))

df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
#df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
#df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']



df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
#df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']

In [19]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description,brand,len_of_query,len_of_title,len_of_description,len_of_brand,product_info,attr,word_in_title,word_in_description,word_in_brand
0,2,simpson strong-ti 12-gaug angl,100001,3.00,angl bracket,"not angl make joint stronger, also provid cons...",simpson strong-ti,2,4,86,2,angl bracket\tsimpson strong-ti 12-gaug angl\t...,angl bracket\tsimpson strong-ti,1,1,0
1,3,simpson strong-ti 12-gaug angl,100001,2.50,l bracket,"not angl make joint stronger, also provid cons...",simpson strong-ti,2,4,86,2,l bracket\tsimpson strong-ti 12-gaug angl\tnot...,l bracket\tsimpson strong-ti,1,1,0
2,9,behr premium textur deckov 1-gal. #sc-141 tugb...,100002,3.00,deck,behr premium textur deckov innov solid color c...,behr premium textur deckov,1,10,122,4,deck\tbehr premium textur deckov 1-gal. #sc-14...,deck\tbehr premium textur deckov,1,1,1
3,16,delta vero 1-handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat bathroom delta vero single-handl shower ...,delta,3,12,67,1,rain shower head\tdelta vero 1-handl shower on...,rain shower head\tdelta,1,1,0
4,17,delta vero 1-handl shower onli faucet trim kit...,100005,2.67,shower faucet,updat bathroom delta vero single-handl shower ...,delta,2,12,67,1,shower faucet\tdelta vero 1-handl shower onli ...,shower faucet\tdelta,2,2,0


In [23]:
df_all_t = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)

In [24]:
df_all_t.head()

,id,product_uid,relevance,brand,len_of_query,len_of_title,len_of_description,len_of_brand,attr,word_in_title,word_in_description,word_in_brand
0,2,100001,3.00,simpson strong-ti,2,4,86,2,angl bracket\tsimpson strong-ti,1,1,0
1,3,100001,2.50,simpson strong-ti,2,4,86,2,l bracket\tsimpson strong-ti,1,1,0
2,9,100002,3.00,behr premium textur deckov,1,10,122,4,deck\tbehr premium textur deckov,1,1,1
3,16,100005,2.33,delta,3,12,67,1,rain shower head\tdelta,1,1,0
4,17,100005,2.67,delta,2,12,67,1,shower faucet\tdelta,2,2,0


In [25]:
df_all_t = df_all_t.drop(['attr','brand'],axis=1)
df_all_t.head()

,id,product_uid,relevance,len_of_query,len_of_title,len_of_description,len_of_brand,word_in_title,word_in_description,word_in_brand
0,2,100001,3.00,2,4,86,2,1,1,0
1,3,100001,2.50,2,4,86,2,1,1,0
2,9,100002,3.00,1,10,122,4,1,1,1
3,16,100005,2.33,3,12,67,1,1,1,0
4,17,100005,2.67,2,12,67,1,2,2,0


In [59]:
df_train = df_all_t.iloc[:num_train]
df_test = df_all_t.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values



In [80]:
df_train.to_csv('df_train.csv',index=False)
df_test.to_csv('df_test.csv',index=False)

pd.DataFrame(y_train).to_csv('y_train.csv')
pd.DataFrame(X_train).to_csv('X_train.csv')
pd.DataFrame(X_test).to_csv('X_test.csv')


In [23]:
#If you are not running whole script for model building

X_train=pd.read_csv("C:/Users/Karthik/Desktop/Coursera/NLP/home depot/X_train.csv")
XX_train=pd.read_csv("C:/Users/Karthik/Desktop/Coursera/NLP/home depot/X_train.csv")
X_test=pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/X_test.csv')
y_train=pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/y_train.csv')
y_train=y_train.drop(['Unnamed: 0'],axis=1)
X_test=X_test.drop(['Unnamed: 0'],axis=1)
X_train=X_train.drop(['Unnamed: 0'],axis=1)
df_test=pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/df_test.csv')
df_train=pd.read_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/df_train.csv')
id_test = df_test['id']

In [4]:
def train_validate_test_split(df, train_percent=.6, seed=2016):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    train = df.iloc[perm[:train_end]]
    test = df.iloc[perm[train_end:]]
    return train, test
train,test=train_validate_test_split(df_train)

In [5]:
train.head()

,id,product_uid,relevance,len_of_query,len_of_title,len_of_description,len_of_brand,word_in_title,word_in_description,word_in_brand
48343,146763,154269,3.00,3,11,77,1,3,3,0
7899,24612,104912,2.33,2,11,81,1,2,2,0
2109,6508,101119,3.00,4,12,186,2,4,4,1
20622,63669,117350,2.67,3,14,71,1,1,1,0
62920,189705,181356,2.33,2,13,190,2,2,2,1


In [14]:
pd.DataFrame(train).to_csv('train.csv')
pd.DataFrame(test).to_csv('test.csv')

In [15]:
df_train.head()

,id,product_uid,relevance,len_of_query,len_of_title,len_of_description,len_of_brand,word_in_title,word_in_description,word_in_brand
0,2,100001,3.00,2,4,86,2,1,1,0
1,3,100001,2.50,2,4,86,2,1,1,0
2,9,100002,3.00,1,10,122,4,1,1,1
3,16,100005,2.33,3,12,67,1,1,1,0
4,17,100005,2.67,2,12,67,1,2,2,0


In [6]:
import pip
pip.main(['install', 'xgboost'])

No files/directories in C:\Users\Karthik\AppData\Local\Temp\pip-build-_qctko5i\xgboost\pip-egg-info (from PKG-INFO)


1

In [15]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as st
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from xgboost.sklearn import XGBClassifier

In [4]:
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}

xgbreg = XGBRegressor()  

In [7]:


gs = RandomizedSearchCV(xgbreg, params, n_jobs=1,return_train_score=True,n_iter=20,cv=10,verbose=1,random_state=2016)  
gs.fit(X_train, y_train)  


Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  4.5min finished


{'colsample_bytree': 0.88654308007704141,
 'gamma': 3.5727324740212554,
 'learning_rate': 0.28867382828053406,
 'max_depth': 24,
 'min_child_weight': 14.557905868858725,
 'n_estimators': 17,
 'reg_alpha': 2.1001384849095248,
 'subsample': 0.94946067493936037}

In [11]:
gs.best_params_

{'colsample_bytree': 0.88654308007704141,
 'gamma': 3.5727324740212554,
 'learning_rate': 0.28867382828053406,
 'max_depth': 24,
 'min_child_weight': 14.557905868858725,
 'n_estimators': 17,
 'reg_alpha': 2.1001384849095248,
 'subsample': 0.94946067493936037}

In [10]:
gs.best_estimator_ 

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.88654308007704141, gamma=3.5727324740212554,
       learning_rate=0.28867382828053406, max_delta_step=0, max_depth=24,
       min_child_weight=14.557905868858725, missing=None, n_estimators=17,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=2.1001384849095248, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.94946067493936037)

In [9]:
gs.best_score_

0.16791861244737541

In [12]:
y_pred = gs.predict(X_test)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)
#score=0.483

In [48]:
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X_train, y_train)  

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_child_weight': [4, 5], 'gamma': [0.3, 0.4, 0.5], 'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [50]:
gs.best_score_

0.16791861244737541

In [54]:
y_pred = gs.predict(X_test)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('C:/Users/Karthik/Desktop/Coursera/NLP/home depot/submission2.csv',index=False)


In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective='reg:linear', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_ 


In [67]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

In [68]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=45, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submissionrf.csv',index=False)
#score .485

In [69]:
gs_rf = RandomizedSearchCV(xgbreg, params, n_jobs=1,return_train_score=True,n_iter=20,cv=10,verbose=1,random_state=2016) 
gs_rf.fit(X_train, y_train)  
gs_rf.best_estimator_ 
gs_rf.best_score_
gs_rf.best_params_

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.9min finished


{'colsample_bytree': 0.86967469608348613,
 'gamma': 3.3770041145740191,
 'learning_rate': 0.41621512691468016,
 'max_depth': 17,
 'min_child_weight': 25.360356047575909,
 'n_estimators': 29,
 'reg_alpha': 9.3489630990652639,
 'subsample': 0.82057968501387568}

In [ ]:
#import h2o
#h2o.init()
#from h2o.estimators.glm import H2OGeneralizedLinearEstimator   # import the glm estimator object
#train2 = h2o.import_file('train.csv')
#test2=h2o.import_file('test.csv')
#train2=train2.drop('C1')
#test2=test2.drop('C1')
#x = train2.columns
#y = "relevance"
#x.remove(y)
#from h2o.automl import H2OAutoML
#aml = H2OAutoML(max_runtime_secs = 30)
#aml.train(x = x, y = y,
#          training_frame = train2,
#          leaderboard_frame = test2)